## Hometask 1 - связь Python с программами на C/C++

1. Реализовать на языке C/C++ классические операции перемножения квадратных матриц и умножения матрицы на вектор (15%)

Решение: см. код в текушей директории:
- файл `matrix_operations` - с операциями над матрицами
- файлы `main_matrix_multiply`, `main_matrix_vector_multiply` - с `main` функциями для экспериментов.

2. Разделить программу на несколько модулей и провести сборку через статическую линковку (25%)

In [5]:
! gcc -c -g matrix_operations.c
! gcc -c -g main_matrix_multiply.c
! gcc matrix_operations.o main_matrix_multiply.o -o main_matrix_multiply_program_g

! gcc -c -O3 matrix_operations.c
! gcc -c -O3 main_matrix_multiply.c
! gcc matrix_operations.o main_matrix_multiply.o -o main_matrix_multiply_program_O3

In [6]:
! gcc -c -g matrix_operations.c
! gcc -c -g main_matrix_vector_multiply.c
! gcc matrix_operations.o main_matrix_vector_multiply.o -o main_matrix_vector_multiply_program_g

! gcc -c -O3 matrix_operations.c
! gcc -c -O3 main_matrix_vector_multiply.c
! gcc matrix_operations.o main_matrix_vector_multiply.o -o main_matrix_vector_multiply_program_O3

3. Подготовьте две сборки с флагами -g и -O3 и измерьте времена выполнения операций с N = 512, 1024, . . ., 4096 (20%).

При данных $N$ машина считает довольно медленно. Уменьшил величины проверяемых значений.

In [12]:
import numpy as np
import os
import pandas as pd
import subprocess

from pathlib import Path


root = Path(os.getcwd())
def run_program(program_name, N):
    subprocess.run([str(root / program_name), str(N)])


programs = {
    'Умножение матриц (-g)': 'main_matrix_multiply_program_g',
    'Умножение матриц (-O3)': 'main_matrix_multiply_program_O3',
    'Умножение матрицы на вектор (-g)': 'main_matrix_vector_multiply_program_g',
    'Умножение матрицы на вектор (-O3)': 'main_matrix_vector_multiply_program_O3',
}

res_c = pd.DataFrame(index=programs.keys())
n_elements = [2**i for i in range(6, 11)]
for n_element in n_elements:
    res_c[n_element] = np.nan
    for experiment_name, program_name in programs.items():
        sum_time = 0
        n_repeat = 4
        for i_repeat in range(n_repeat):
            start = time.perf_counter()
            run_program(program_name, n_element)
            end = time.perf_counter()
            sum_time += end - start
        res_c.loc[experiment_name, n_element] = sum_time / n_repeat
        
res_c

,64,128,256,512,1024
Умножение матриц (-g),0.005413,0.013333,0.079339,0.607887,5.653850
Умножение матриц (-O3),0.004153,0.005167,0.023636,0.197036,1.756253
Умножение матрицы на вектор (-g),0.004011,0.002818,0.003560,0.004528,0.009585
Умножение матрицы на вектор (-O3),0.003551,0.002710,0.003362,0.003546,0.005921


4. Выполните вызов процедуры из Python через Ctypes/Cython/PyBind11 и измерьте времена (40%)

Получилось реализовать только Ctypes:

In [8]:
!gcc -O3 -shared matrix_operations.o -o matrix_operations.so

In [9]:
import ctypes
import numpy as np
import time


def run_ctypes_matrix_multiply(n):
    lib = ctypes.CDLL('./matrix_operations.so')
    
    matrix_multiply = lib.matrix_multiply
    matrix_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
    matrix_multiply.restype = None
    
    A = np.random.rand(n, n)
    B = np.random.rand(n, n)
    
    C = np.empty((n, n), dtype=np.double)
    matrix_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                    B.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                    ctypes.c_int(n),
                    C.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))
    
    return C
    

def run_ctypes_matrix_vector_multiply(n):
    lib = ctypes.CDLL('./matrix_operations.so')
    
    matrix_vector_multiply = lib.matrix_vector_multiply
    matrix_vector_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
    matrix_vector_multiply.restype = None
    
    A = np.ones((n, n), dtype=np.double)
    x = np.full(n, 2.0, dtype=np.double)
    y = np.empty(n, dtype=np.double)
    
    matrix_vector_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                      x.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                      ctypes.c_int(n),
                      y.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))
    
    return y
    

programs = {
    'Умножение матриц (Ctypes)': run_ctypes_matrix_multiply,
    'Умножение матрицы на вектор (Ctypes)': run_ctypes_matrix_vector_multiply,
}

res_ctypes = pd.DataFrame(index=programs.keys())
for experiment_name, func in programs.items():
    n_repeat = 5
    for n_element in n_elements + [2048]:
        sum_time = 0
        for i_repeat in range(n_repeat):
            start = time.perf_counter()
            func(n_element)
            end = time.perf_counter()
            sum_time += end - start
        res_ctypes.loc[experiment_name, n_element] = sum_time / n_repeat

Проверим корректность вычисления:

In [10]:
lib = ctypes.CDLL('./matrix_operations.so')
    
matrix_multiply = lib.matrix_multiply
matrix_multiply.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.c_int, ctypes.POINTER(ctypes.c_double)]
matrix_multiply.restype = None

A = np.array([[1, 2], 
              [3, 2]], dtype=np.double)
B = np.array([[1, 1], 
              [3, 2]], dtype=np.double)

C = np.empty((2, 2), dtype=np.double)
matrix_multiply(A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                B.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
                ctypes.c_int(2),
                C.ctypes.data_as(ctypes.POINTER(ctypes.c_double)))

C

array([[7., 5.],
       [9., 7.]])

Вычисление проходит корректно.

# Результаты в сравнении

In [13]:
pd.concat([res_c, res_ctypes])

,64,128,256,512,1024,2048
Умножение матриц (-g),0.005413,0.013333,0.079339,0.607887,5.653850,NaN
Умножение матриц (-O3),0.004153,0.005167,0.023636,0.197036,1.756253,NaN
Умножение матрицы на вектор (-g),0.004011,0.002818,0.003560,0.004528,0.009585,NaN
Умножение матрицы на вектор (-O3),0.003551,0.002710,0.003362,0.003546,0.005921,NaN
Умножение матриц (Ctypes),0.054663,0.002489,0.021480,0.204413,1.859328,30.296079
Умножение матрицы на вектор (Ctypes),0.000112,0.000203,0.000128,0.000577,0.002303,0.010021


In [7]:
pd.concat([res_c, res_ctypes])

,64,128,256,512,1024,2048
Умножение матриц (-g),0.436641,0.049266,0.246818,1.871151,18.098031,NaN
Умножение матриц (-O3),0.191084,0.018367,0.076522,0.605391,5.811974,NaN
Умножение матрицы на вектор (-g),0.206288,0.011110,0.011465,0.015858,0.035111,NaN
Умножение матрицы на вектор (-O3),0.273246,0.011456,0.011290,0.011774,0.024650,NaN
Умножение матриц (Ctypes),0.055312,0.004103,0.021261,0.154889,1.449117,27.274883
Умножение матрицы на вектор (Ctypes),0.000073,0.000126,0.000122,0.000450,0.001858,0.008314


На самом деле, меня смущает такое расхождение по времени между Ctypes и версией на С, хотя время прирастает в соответствии со сложностью алгоритмов.